In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from scipy import stats
from sklearn import linear_model
from sklearn.metrics import mean_squared_error
from sklearn import metrics
import scipy
from sklearn import preprocessing

In [2]:
fmli = pd.read_csv("fmli_15_clean.csv")
fmli.drop(["DETTACHED"], axis = 1, inplace = True)

In [3]:
sf = pd.read_csv("census.tracts.sf.cleaned.csv")

In [4]:
sf.drop(sf.columns[[2, 4, 6, 8, 10, 12, 14]], axis = 1, inplace = True)
sf.dropna(inplace = True)

In [5]:
pred_X = sf.drop(sf.columns[[0, 1]], axis = 1)

In [6]:
sf.head()

,CT.Geo_GEOID,CT.Geo_NAME,CT.AVGINCOME,CT.SIZE,CT.ROOMS,CT.DEGREE,CT.OWN,CT.VEHICLES
0,14000US06075010100,"Census Tract 101, San Francisco County, Califo...",99091.891892,1.842752,3.184542,0.596069,0.099263,0.635381
1,14000US06075010200,"Census Tract 102, San Francisco County, Califo...",135164.337061,1.710064,4.223822,0.798323,0.363818,0.991613
2,14000US06075010300,"Census Tract 103, San Francisco County, Califo...",124997.527201,2.054896,4.320723,0.656281,0.263106,0.962413
3,14000US06075010400,"Census Tract 104, San Francisco County, Califo...",132752.166735,1.865043,4.038502,0.782914,0.362361,0.883615
4,14000US06075010500,"Census Tract 105, San Francisco County, Califo...",123030.754597,1.624604,3.093623,0.771084,0.187064,0.629677


In [7]:
pred_X = sf[sf.columns[[3, 2, 4, 6, 5, 7]]]

##### Normalize data frame

In [8]:
# Log every variables, save it in a new data frame for future use.
log = np.log(fmli)
log["OWN"] = fmli["OWN"]
log["DEGREE"] = fmli["DEGREE"]
log["VEHICLES"] = fmli["VEHICLES"]

/Users/winnie/miniconda2/envs/tf/lib/python3.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning: divide by zero encountered in log
  
/Users/winnie/miniconda2/envs/tf/lib/python3.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in log
  


In [18]:
# Log every variables, save it in a new data frame for future use.
log_X = np.log(pred_X)
log_X["CT.OWN"] = pred_X["CT.OWN"]
log_X["CT.DEGREE"] = pred_X["CT.DEGREE"]
log_X["CT.VEHICLES"] = pred_X["CT.VEHICLES"]
log_X.head()

/Users/winnie/miniconda2/envs/tf/lib/python3.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning: divide by zero encountered in log
  


,CT.SIZE,CT.AVGINCOME,CT.ROOMS,CT.OWN,CT.DEGREE,CT.VEHICLES
0,0.611260,11.503803,1.158308,0.099263,0.596069,0.635381
1,0.536531,11.814247,1.440740,0.363818,0.798323,0.991613
2,0.720225,11.736049,1.463423,0.263106,0.656281,0.962413
3,0.623284,11.796239,1.395874,0.362361,0.782914,0.883615
4,0.485264,11.720190,1.129343,0.187064,0.771084,0.629677


##### Multivariable Model: HHSIZE + INCOME + ROOMS  + OWN + DEGREE + VEH

In [33]:
N = []
adj_r = []
b_hhsize = []
b_income = []
b_rooms = []
b_own = []
b_degree = []
b_veh = []
se_hhsize = []
se_income = []
se_rooms = []
se_own = []
se_degree = []
se_veh = []
p_hhsize = []
p_income = []
p_rooms = []
p_own = []
p_degree = []
p_veh = []

for i in range(16, 17):
    df = log[["HHSIZE", "INCOME", "ROOMS", "OWN", "DEGREE", "VEHICLES", log.columns[i]]]
    df.replace(np.Inf, np.nan, inplace = True)
    df.replace(-np.Inf, np.nan, inplace = True)
    df.dropna(inplace = True)
    df.reset_index(inplace = True)
    X = df[["HHSIZE", "INCOME", "ROOMS", "OWN", "DEGREE", "VEHICLES"]]
    Y = df[log.columns[i]]
    X1 = sm.add_constant(X)
    model = sm.OLS(Y, X1).fit()
    N.append(len(Y))
    b_hhsize.append(model.params[1])
    b_income.append(model.params[2])
    b_rooms.append(model.params[3])
    b_own.append(model.params[4])
    b_degree.append(model.params[5])
    b_veh.append(model.params[6])
    adj_r.append(model.rsquared_adj)
    se_hhsize.append(model.bse[1])
    se_income.append(model.bse[2])
    se_rooms.append(model.bse[3])
    se_own.append(model.bse[4])
    se_degree.append(model.bse[5])
    se_veh.append(model.bse[6])
    p_hhsize.append(model.pvalues[1])
    p_income.append(model.pvalues[2])
    p_rooms.append(model.pvalues[3])
    p_own.append(model.pvalues[4])
    p_degree.append(model.pvalues[5])
    p_veh.append(model.pvalues[6])

/Users/winnie/miniconda2/envs/tf/lib/python3.7/site-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


##### Second model with Lasso dropping insignificant variables

In [11]:
N = []
adj_r = []
b_hhsize = []
b_income = []
b_rooms = []
b_own = []
b_degree = []
b_veh = []
se_hhsize = []
se_income = []
se_rooms = []
se_own = []
se_degree = []
se_veh = []
p_hhsize = []
p_income = []
p_rooms = []
p_own = []
p_degree = []
p_veh = []
pred_y = pd.DataFrame()

for i in range(16, len(log.columns)):
    df = log[["HHSIZE", "INCOME", "ROOMS", "OWN", "DEGREE", "VEHICLES", log.columns[i]]]
    df.replace(np.Inf, np.nan, inplace = True)
    df.replace(-np.Inf, np.nan, inplace = True)
    df.dropna(inplace = True)
    df.reset_index(inplace = True)
    X = df[["HHSIZE", "INCOME", "ROOMS", "OWN", "DEGREE", "VEHICLES"]]
    Y = df[log.columns[i]]
    X1 = sm.add_constant(X)
    model = linear_model.Lasso(alpha=0.03).fit(X1, Y)
    log_X1 = sm.add_constant(log_X)
    pred_y[log.columns[i]] = np.array(model.predict(log_X1))
    N.append(len(Y))
    b_hhsize.append(model.coef_[1])
    b_income.append(model.coef_[2])
    b_rooms.append(model.coef_[3])
    b_own.append(model.coef_[4])
    b_degree.append(model.coef_[5])
    b_veh.append(model.coef_[6])
    

    new = pd.DataFrame()
    if model.coef_[1] != 0:
        new["HHSIZE"] = df["HHSIZE"]
    if model.coef_[2] != 0:
        new["INCOME"] = df["INCOME"]
    if model.coef_[3] != 0:
        new["ROOMS"] = df["ROOMS"]
    if model.coef_[4] != 0:
        new["OWN"] = df["OWN"]
    if model.coef_[5] != 0:
        new["DEGREE"] = df["DEGREE"]
    if model.coef_[6] != 0:
        new["VEHICLES"] = df["VEHICLES"]
    new = sm.add_constant(new)
    model1 = sm.OLS(Y, new).fit() 
    adj_r.append(model1.rsquared_adj)
    bse = list(model1.bse)
    bse.pop(0)
    p = list(model1.pvalues)
    p.pop(0)
    if model.coef_[1] != 0:
        se_hhsize.append(bse.pop(0))
        p_hhsize.append(p.pop(0))
    else:
        se_hhsize.append(-1)
        p_hhsize.append(-1)
    if model.coef_[2] != 0:
        se_income.append(bse.pop(0))
        p_income.append(p.pop(0))
    else:
        se_income.append(-1)
        p_income.append(-1)
    if model.coef_[3] != 0:
        se_rooms.append(bse.pop(0))
        p_rooms.append(p.pop(0))
    else:
        se_rooms.append(-1)
        p_rooms.append(-1)
    if model.coef_[4] != 0:
        se_own.append(bse.pop(0))
        p_own.append(p.pop(0))
    else:
        se_own.append(-1)
        p_own.append(-1)
    if model.coef_[5] != 0:
        se_degree.append(bse.pop(0))
        p_degree.append(p.pop(0))
    else:
        se_degree.append(-1)
        p_degree.append(-1)
    if model.coef_[6] != 0:
        se_veh.append(bse.pop(0))
        p_veh.append(p.pop(0))
    else:
        se_veh.append(-1)
        p_veh.append(-1)

/Users/winnie/miniconda2/envs/tf/lib/python3.7/site-packages/pandas/core/frame.py:4042: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  method=method)
/Users/winnie/miniconda2/envs/tf/lib/python3.7/site-packages/ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/winnie/miniconda2/envs/tf/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
/Users/winnie/miniconda2/envs/tf/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and w

/Users/winnie/miniconda2/envs/tf/lib/python3.7/site-packages/ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/winnie/miniconda2/envs/tf/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
/Users/winnie/miniconda2/envs/tf/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
/Users/winnie/miniconda2/envs/tf/lib/python3.7/site-packages/pandas/core/frame.py:4042: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation:

/Users/winnie/miniconda2/envs/tf/lib/python3.7/site-packages/ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/winnie/miniconda2/envs/tf/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
/Users/winnie/miniconda2/envs/tf/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
/Users/winnie/miniconda2/envs/tf/lib/python3.7/site-packages/pandas/core/frame.py:4042: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation:

/Users/winnie/miniconda2/envs/tf/lib/python3.7/site-packages/pandas/core/frame.py:4042: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  method=method)
/Users/winnie/miniconda2/envs/tf/lib/python3.7/site-packages/ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/winnie/miniconda2/envs/tf/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
/Users/winnie/miniconda2/envs/tf/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and w

/Users/winnie/miniconda2/envs/tf/lib/python3.7/site-packages/ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/winnie/miniconda2/envs/tf/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
/Users/winnie/miniconda2/envs/tf/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
/Users/winnie/miniconda2/envs/tf/lib/python3.7/site-packages/pandas/core/frame.py:4042: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation:

/Users/winnie/miniconda2/envs/tf/lib/python3.7/site-packages/ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/winnie/miniconda2/envs/tf/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
/Users/winnie/miniconda2/envs/tf/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
/Users/winnie/miniconda2/envs/tf/lib/python3.7/site-packages/pandas/core/frame.py:4042: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation:

/Users/winnie/miniconda2/envs/tf/lib/python3.7/site-packages/ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/winnie/miniconda2/envs/tf/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
/Users/winnie/miniconda2/envs/tf/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
/Users/winnie/miniconda2/envs/tf/lib/python3.7/site-packages/pandas/core/frame.py:4042: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation:

In [16]:
result = pred_y
for i in pred_y.columns:
    result[i] = np.exp(pred_y[i])

In [17]:
result

,TotalExpenditures,TotalFood,FoodAtHome,FoodAwayFromHome,AlcoholicBeverages,Housing,Shelter,OtherLodging,UtilitiesFuelsAndPublicServices,TelephoneServices,...,PetsToysAndPlaygroundEquipment,OtherEntertainment,PersonalCare,Reading,Education,TobaccoAndSmokingSupplies,MiscellaneousExpenditures,CashContributions,PersonalInsuranceAndPensions,AdjustedTotalExpenditures
0,26443.329594,4025.175808,2581.391072,1401.517195,331.281924,9466.709367,6942.295049,1491.088510,341.748700,669.146424,...,429.819755,394.974491,210.854394,194.896926,1593.873618,761.902174,259.434859,763.618501,3675.411797,26443.329594
1,30901.612986,4211.762977,2571.873182,1532.795973,364.626358,10354.610608,7008.626655,1637.349275,411.653971,702.226207,...,475.295260,437.169118,229.732935,206.467524,2025.747292,782.064854,281.778084,970.693769,4865.332270,30901.612986
2,30491.710456,4400.284263,2781.717686,1509.926653,350.085477,10300.842659,7053.925294,1574.927631,421.245433,726.559799,...,467.095641,431.156834,230.187110,204.941828,1776.963951,782.527889,278.516836,925.053622,4557.725576,30491.710456
3,30747.796453,4313.059416,2669.387922,1533.853239,360.091178,10461.698958,7031.416272,1627.397079,411.155274,710.924823,...,470.119508,430.175481,230.612559,205.290278,1923.238050,780.625947,279.162858,946.308192,4800.377438,30747.796453
4,28090.797842,4018.812590,2478.483113,1476.146978,354.243447,9963.549708,7336.754087,1599.697824,340.060462,667.120093,...,446.510783,407.837215,217.776568,199.196051,1845.286331,767.595861,268.077226,825.053297,4341.387232,28090.797842
5,23467.539784,3799.941744,2517.106318,1293.290570,306.892505,8695.598040,6073.690780,1380.732775,328.010898,635.388926,...,405.767716,368.154941,197.842972,188.486627,1399.815908,744.679406,245.758341,688.072869,2934.763562,23467.539784
6,21070.453314,3735.753309,2608.121290,1190.909819,278.322323,7985.477239,5295.427431,1245.078123,321.034049,627.559112,...,378.130977,347.450822,185.221339,180.373351,1147.274410,734.524919,231.062156,602.685191,2263.647150,21070.453314
7,29228.787895,4184.803853,2622.948337,1477.501953,348.308830,10035.177271,7042.829426,1571.068930,396.769129,693.774666,...,460.023896,419.210646,224.479353,203.020715,1813.813596,773.077506,273.741913,898.804816,4314.766040,29228.787895
8,29858.092801,4113.960169,2513.293093,1512.963866,362.317279,10219.078333,7495.135084,1632.345660,380.102174,685.014137,...,468.380242,426.122843,226.786881,205.410612,1966.098570,775.195268,278.784257,918.734467,4709.888303,29858.092801
9,29752.171637,4288.219816,2670.952115,1524.535376,358.163085,10413.960465,7589.470029,1623.060274,374.917282,703.699984,...,462.404078,419.591251,228.981014,204.510268,1803.475671,775.850857,277.252039,878.120266,4772.059146,29752.171637


In [71]:
pred_y.to_csv("pred_y.csv")

In [60]:
b_hhsize = [round(x, 3) for x in b_hhsize]
b_income = [round(x, 3) for x in b_income]
b_rooms = [round(x, 3) for x in b_rooms]
b_own = [round(x, 3) for x in b_own]
b_degree = [round(x, 3) for x in b_degree]
b_veh = [round(x, 3) for x in b_veh]

se_hhsize = [round(x, 3) for x in se_hhsize]
se_income = [round(x, 3) for x in se_income]
se_rooms = [round(x, 3) for x in se_rooms]
se_own = [round(x, 3) for x in se_own]
se_degree = [round(x, 3) for x in se_degree]
se_veh = [round(x, 3) for x in se_veh]

p_hhsize = [round(x, 3) for x in p_hhsize]
p_income = [round(x, 3) for x in p_income]
p_rooms = [round(x, 3) for x in p_rooms]
p_own = [round(x, 3) for x in p_own]
p_degree = [round(x, 3) for x in p_degree]
p_veh = [round(x, 3) for x in p_veh]

N = [round(x, 0) for x in N]
adj_r = [round(x, 3) for x in adj_r]

In [61]:
index = []
for i in range(len(log.columns[16: len(log.columns)].values)):
    index.append(log.columns[16: len(log.columns)].values[i])
for i in range(len(log.columns[16: len(log.columns)].values)):
    index.append(log.columns[16: len(log.columns)].values[i])
for i in range(len(log.columns[16: len(log.columns)].values)):
    index.append(log.columns[16: len(log.columns)].values[i])

In [62]:
for i in range(len(b_hhsize)):
    if b_hhsize[i] == 0:
        b_hhsize[i] = " "
for i in range(len(b_income)):
    if b_income[i] == 0:
        b_income[i] = " "
for i in range(len(b_rooms)):
    if b_rooms[i] == 0:
        b_rooms[i] = " "
for i in range(len(b_own)):
    if b_own[i] == 0:
        b_own[i] = " "
for i in range(len(b_degree)):
    if b_degree[i] == 0:
        b_degree[i] = " "
for i in range(len(b_veh)):
    if b_veh[i] == 0:
        b_veh[i] = " "
        
for i in range(len(se_hhsize)):
    if se_hhsize[i] < 0:
        se_hhsize[i] = " "
for i in range(len(se_income)):
    if se_income[i] < 0:
        se_income[i] = " "
for i in range(len(se_rooms)):
    if se_rooms[i] < 0:
        se_rooms[i] = " "
for i in range(len(se_own)):
    if se_own[i] < 0:
        se_own[i] = " "
for i in range(len(se_degree)):
    if se_degree[i] < 0:
        se_degree[i] = " "
for i in range(len(se_veh)):
    if se_veh[i] < 0:
        se_veh[i] = " "
        
for i in range(len(p_hhsize)):
    if p_hhsize[i] < 0:
        p_hhsize[i] = " "
for i in range(len(p_income)):
    if p_income[i] < 0:
        p_income[i] = " "
for i in range(len(p_rooms)):
    if p_rooms[i] < 0:
        p_rooms[i] = " "
for i in range(len(p_own)):
    if p_own[i] < 0:
        p_own[i] = " "
for i in range(len(p_degree)):
    if p_degree[i] < 0:
        p_degree[i] = " "
for i in range(len(p_veh)):
    if p_veh[i] < 0:
        p_veh[i] = " "

In [63]:
len(p_degree)

54

In [64]:
parameter = pd.DataFrame({"parameter": ["Beta"] * 54 + ["SE"] * 54 + ["Confidence Level"] * 54,
                          "HHSIZE": np.concatenate([b_hhsize, se_hhsize, p_hhsize]),
                          "INCOME": np.concatenate([b_income, se_income, p_income]),
                          "ROOMS": np.concatenate([b_rooms, se_rooms, p_rooms]),
                          "OWN": np.concatenate([b_own, se_own, p_own]),
                          "DEGREE": np.concatenate([b_degree, se_degree, p_degree]),
                          "VEH": np.concatenate([b_veh, se_veh, p_veh]),
                          "N": N + [" "] * 54 + ["  "] * 54,
                          "adj_r2": adj_r + [" "] * 54 + ["  "] * 54,
                          "index": index})

In [65]:
parameter = pd.DataFrame(parameter.groupby(["index", "N", "adj_r2", "parameter", "HHSIZE", "INCOME", "ROOMS", "OWN", "DEGREE", "VEH"]).apply(list)).drop([0], axis = 1)

In [66]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    display(parameter)

Empty DataFrame
Columns: []
Index: [(AdjustedTotalExpenditures, 26855, 0.36, Beta, 0.175, 0.249, 0.092,  , 0.181, 0.081), (AdjustedTotalExpenditures,  ,  , SE, 0.008, 0.004, 0.012,  , 0.009, 0.003), (AdjustedTotalExpenditures,   ,   , Confidence Level, 0.0, 0.0, 0.0,  , 0.0, 0.0), (AlcoholicBeverages, 12960, 0.06, Beta, -0.049, 0.192,  ,  , 0.112, 0.029), (AlcoholicBeverages,  ,  , SE, 0.021, 0.01,  ,  , 0.023, 0.007), (AlcoholicBeverages,   ,   , Confidence Level, 0.0, 0.0,  ,  , 0.0, 0.0), (ApparelAndServices, 16365, 0.12, Beta, 0.385, 0.204, 0.086,  , 0.047, 0.018), (ApparelAndServices,  ,  , SE, 0.019, 0.009, 0.027,  , 0.022, 0.007), (ApparelAndServices,   ,   , Confidence Level, 0.0, 0.0, 0.0,  , 0.0, 0.155), (BabysittingAndChildDayCare, 1716, 0.059, Beta,  , 0.209,  ,  , 0.127, -0.018), (BabysittingAndChildDayCare,  ,  , SE,  , 0.028,  ,  , 0.062, 0.023), (BabysittingAndChildDayCare,   ,   , Confidence Level,  , 0.0,  ,  , 0.0, 0.065), (CarsAndTrucksNewNetOutlay, 285, 0.018, Beta,  , 0.028,  ,  ,  , 0.016), (CarsAndTrucksNewNetOutlay,  ,  , SE,  , 0.024,  ,  ,  , 0.017), (CarsAndTrucksNewNetOutlay,   ,   , Confidence Level,  , 0.051,  ,  ,  , 0.157), (CarsAndTrucksUsedNetOutlay, 746, 0.068, Beta,  , 0.255, 0.099, 0.07, 0.107, -0.036), (CarsAndTrucksUsedNetOutlay,  ,  , SE,  , 0.051, 0.144, 0.116, 0.103, 0.029), (CarsAndTrucksUsedNetOutlay,   ,   , Confidence Level,  , 0.0, 0.061, 0.14, 0.04, 0.022), (CashContributions, 12474, 0.095, Beta,  , 0.213, 0.307, 0.143, 0.16, 0.048), (CashContributions,  ,  , SE,  , 0.012, 0.041, 0.035, 0.028, 0.009), (CashContributions,   ,   , Confidence Level,  , 0.0, 0.0, 0.0, 0.0, 0.0), (DomesticServices, 7317, 0.184, Beta, 0.394, 0.221, 0.829, 0.024, 0.2, -0.015), (DomesticServices,  ,  , SE, 0.036, 0.016, 0.055, 0.05, 0.039, 0.013), (DomesticServices,   ,   , Confidence Level, 0.0, 0.0, 0.0, 0.001, 0.0, 0.0), (DomesticServicesExclChildcare, 6112, 0.221, Beta, -0.044, 0.173, 0.981, 0.411, 0.126,  ), (DomesticServicesExclChildcare,  ,  , SE, 0.035, 0.015, 0.053, 0.05, 0.038,  ), (DomesticServicesExclChildcare,   ,   , Confidence Level, 0.0, 0.0, 0.0, 0.0, 0.0,  ), (Education, 2609, 0.081, Beta, -0.334, 0.207, 0.126, 0.082, 0.322, 0.079), (Education,  ,  , SE, 0.067, 0.031, 0.101, 0.086, 0.071, 0.023), (Education,   ,   , Confidence Level, 0.0, 0.0, 0.001, 0.079, 0.0, 0.0), (Entertainment, 23419, 0.134, Beta, 0.049, 0.186, 0.198, 0.052, 0.156, 0.061), (Entertainment,  ,  , SE, 0.014, 0.006, 0.022, 0.018, 0.015, 0.005), (Entertainment,   ,   , Confidence Level, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0), (FeesAndAdmissions, 9453, 0.124, Beta, 0.194, 0.199, 0.123, 0.119, 0.317, 0.006), (FeesAndAdmissions,  ,  , SE, 0.026, 0.012, 0.04, 0.034, 0.027, 0.009), (FeesAndAdmissions,   ,   , Confidence Level, 0.0, 0.0, 0.0, 0.0, 0.0, 0.762), (FloorCoverings, 822, 0.089, Beta,  , 0.104, 0.214, 0.796,  , 0.003), (FloorCoverings,  ,  , SE,  , 0.054, 0.179, 0.159,  , 0.035), (FloorCoverings,   ,   , Confidence Level,  , 0.061, 0.046, 0.0,  , 0.93), (FoodAtHome, 26603, 0.22, Beta, 0.465, 0.083,  ,  ,  , 0.015), (FoodAtHome,  ,  , SE, 0.008, 0.004,  ,  ,  , 0.003), (FoodAtHome,   ,   , Confidence Level, 0.0, 0.0,  ,  ,  , 0.0), (FoodAwayFromHome, 21720, 0.117, Beta, 0.106, 0.199,  ,  , 0.129, 0.028), (FoodAwayFromHome,  ,  , SE, 0.013, 0.006,  ,  , 0.014, 0.005), (FoodAwayFromHome,   ,   , Confidence Level, 0.0, 0.0,  ,  , 0.0, 0.0), (Furniture, 2409, 0.041, Beta,  , 0.176,  , 0.216,  ,  ), (Furniture,  ,  , SE,  , 0.027,  , 0.066,  ,  ), (Furniture,   ,   , Confidence Level,  , 0.0,  , 0.0,  ,  ), (HealthInsurance, 19190, 0.08, Beta, 0.159, 0.067, 0.114, 0.15,  , 0.032), (HealthInsurance,  ,  , SE, 0.015, 0.006, 0.022, 0.019,  , 0.005), (HealthInsurance,   ,   , Confidence Level, 0.0, 0.0, 0.0, 0.0,  , 0.001), (Healthcare, 21357, 0.11, Beta, 0.033, 0.131, 0.207, 0.308, 0.024, 0.06), (Healthcare,  ,  , SE, 0.017, 0.007, 0.025, 0.021, 0.017, 0.006), (Healthcare,   ,   , Confidence Level, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0), (H

In [70]:
parameter.to_csv("multimodel_7var.csv")

##### Normalized multivariable with insignificant variables dropped

In [21]:
N = []
adj_r = []
b_hhsize = []
b_income = []
b_rooms = []
b_own = []
b_degree = []
b_veh = []
se_hhsize = []
se_income = []
se_rooms = []
se_own = []
se_degree = []
se_veh = []
p_hhsize = []
p_income = []
p_rooms = []
p_own = []
p_degree = []
p_veh = []

for i in range(16, len(log.columns)):
    df = log[["HHSIZE", "INCOME", "ROOMS", "OWN", "DEGREE", "VEHICLES", log.columns[i]]]
    df.replace(np.Inf, np.nan, inplace = True)
    df.replace(-np.Inf, np.nan, inplace = True)
    df.dropna(inplace = True)
    df.reset_index(inplace = True)
    X = df[["HHSIZE", "INCOME", "ROOMS", "OWN", "DEGREE", "VEHICLES"]]
    X 
    Y = df[log.columns[i]]
    X1 = sm.add_constant(X)
    model = linear_model.Lasso(alpha=0.001, normalize = True).fit(X1, Y)
    N.append(len(Y))
    b_hhsize.append(model.coef_[1])
    b_income.append(model.coef_[2])
    b_rooms.append(model.coef_[3])
    b_own.append(model.coef_[4])
    b_degree.append(model.coef_[5])
    b_veh.append(model.coef_[6])
    

    new = pd.DataFrame()
    if model.coef_[1] != 0:
        new["HHSIZE"] = df["HHSIZE"]
    if model.coef_[2] != 0:
        new["INCOME"] = df["INCOME"]
    if model.coef_[3] != 0:
        new["ROOMS"] = df["ROOMS"]
    if model.coef_[4] != 0:
        new["OWN"] = df["OWN"]
    if model.coef_[5] != 0:
        new["DEGREE"] = df["DEGREE"]
    if model.coef_[6] != 0:
        new["VEHICLES"] = df["VEHICLES"]
    normalized_new =(new - new.mean()) / new.std()
    normalized_new = sm.add_constant(new)
    model1 = sm.OLS(Y, normalized_new).fit() 
    adj_r.append(model1.rsquared_adj)
    bse = list(model1.bse)
    bse.pop(0)
    p = list(model1.pvalues)
    p.pop(0)
    if model.coef_[1] != 0:
        se_hhsize.append(bse.pop(0))
        p_hhsize.append(p.pop(0))
    else:
        se_hhsize.append(-1)
        p_hhsize.append(-1)
    if model.coef_[2] != 0:
        se_income.append(bse.pop(0))
        p_income.append(p.pop(0))
    else:
        se_income.append(-1)
        p_income.append(-1)
    if model.coef_[3] != 0:
        se_rooms.append(bse.pop(0))
        p_rooms.append(p.pop(0))
    else:
        se_rooms.append(-1)
        p_rooms.append(-1)
    if model.coef_[4] != 0:
        se_own.append(bse.pop(0))
        p_own.append(p.pop(0))
    else:
        se_own.append(-1)
        p_own.append(-1)
    if model.coef_[5] != 0:
        se_degree.append(bse.pop(0))
        p_degree.append(p.pop(0))
    else:
        se_degree.append(-1)
        p_degree.append(-1)
    if model.coef_[6] != 0:
        se_veh.append(bse.pop(0))
        p_veh.append(p.pop(0))
    else:
        se_veh.append(-1)
        p_veh.append(-1)

/Users/winnie/miniconda2/envs/tf/lib/python3.7/site-packages/pandas/core/frame.py:4042: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  method=method)
/Users/winnie/miniconda2/envs/tf/lib/python3.7/site-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/winnie/miniconda2/envs/tf/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
/Users/winnie/miniconda2/envs/tf/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and w

/Users/winnie/miniconda2/envs/tf/lib/python3.7/site-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/winnie/miniconda2/envs/tf/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
/Users/winnie/miniconda2/envs/tf/lib/python3.7/site-packages/pandas/core/frame.py:4042: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  method=method)
/Users/winnie/miniconda2/envs/tf/lib/python3.7/site-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set

ValueError: zero-size array to reduction operation maximum which has no identity

,HHSIZE,INCOME,ROOMS,OWN,DEGREE,VEHICLES
0,0.000000,10.915088,1.609438,0,0,1
1,0.000000,10.714418,1.609438,1,0,1
2,1.386294,9.260082,1.791759,0,0,1
3,0.000000,11.378925,1.609438,1,1,1
4,1.386294,10.915088,1.791759,0,1,3
5,0.693147,11.050557,1.098612,0,0,2
6,1.791759,10.308953,1.945910,1,0,1
7,0.000000,11.082143,1.386294,0,1,1
8,0.693147,10.558881,1.791759,1,0,2
9,0.000000,11.002100,1.609438,1,1,1


In [19]:
model.coef_

array([0.        , 0.074856  , 0.19587623, 0.07541131, 0.        ,
       0.03043274, 0.01845024])

In [26]:
pred_y.mean(axis = 0)

TotalExpenditures                       28188.726650
TotalFood                               16761.965809
FoodAtHome                               9388.939454
FoodAwayFromHome                        22461.523792
AlcoholicBeverages                      21667.590140
Housing                                 23133.892598
Shelter                                 29110.472859
OtherLodging                            21091.560775
UtilitiesFuelsAndPublicServices          8660.430607
TelephoneServices                       15324.306282
WaterAndOtherPublicServices              4184.860880
HouseholdOperations                     19086.153160
DomesticServices                        25023.392668
DomesticServicesExclChildcare           19547.075284
BabysittingAndChildDayCare              23619.701741
OtherhouseHoldExpenses                   7992.619037
HouseFurnishingsAndEquipment            17309.760163
HouseholdTextiles                       17866.047672
Furniture                               19961.